# Preprocess NIH X-Ray Dataset

In [1]:
import pandas as pd

metadata = pd.read_csv("/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/Data_Entry_2017.csv")

metadata

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,058Y,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,058Y,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,058Y,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,081Y,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,081Y,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,039Y,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,029Y,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,042Y,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,030Y,F,PA,2048,2500,0.168,0.168,NaN


In [2]:
class2images = {}

for index, row in metadata.iterrows():
    image = row["Image Index"]
    for label in row["Finding Labels"].split("|"):
        class2images.setdefault(label, []).append(image)

In [6]:
class2images.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation'])

In [4]:
# get rid of the "No Finding" class
class2images.pop("No Finding")

['00000002_000.png',
 '00000005_000.png',
 '00000005_001.png',
 '00000005_002.png',
 '00000005_003.png',
 '00000005_004.png',
 '00000005_005.png',
 '00000006_000.png',
 '00000007_000.png',
 '00000008_001.png',
 '00000011_001.png',
 '00000011_002.png',
 '00000011_003.png',
 '00000011_004.png',
 '00000011_008.png',
 '00000013_000.png',
 '00000013_008.png',
 '00000013_014.png',
 '00000013_015.png',
 '00000013_016.png',
 '00000013_017.png',
 '00000013_019.png',
 '00000013_038.png',
 '00000014_000.png',
 '00000015_000.png',
 '00000016_000.png',
 '00000017_001.png',
 '00000017_002.png',
 '00000018_000.png',
 '00000022_000.png',
 '00000023_000.png',
 '00000023_001.png',
 '00000023_003.png',
 '00000029_000.png',
 '00000031_000.png',
 '00000032_002.png',
 '00000032_003.png',
 '00000032_005.png',
 '00000032_008.png',
 '00000032_009.png',
 '00000032_010.png',
 '00000032_019.png',
 '00000032_020.png',
 '00000032_022.png',
 '00000032_025.png',
 '00000032_042.png',
 '00000032_046.png',
 '00000032_04

In [5]:
class2images.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation'])

In [8]:
train_images = []
with open("/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/train_val_list_NIH.txt") as f:
    for line in f:
        train_images.append(line.strip())

# split train_images into train and val
import random

random.seed(42)

random.shuffle(train_images)

print(len(train_images))
split = int(0.8 * len(train_images))

val_images = train_images[split:]
train_images = train_images[:split]

86524


In [9]:
test_images = []
with open("/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/test_list_NIH.txt") as f:
    for line in f:
        test_images.append(line.strip())

len(train_images), len(val_images), len(test_images)

(69219, 17305, 25596)

In [10]:
class2images.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation'])

In [11]:
# rename Pleural_Thickening to Pleural Thickening
class2images["Pleural Thickening"] = class2images.pop("Pleural_Thickening")

In [14]:
class2images_train = {}
class2images_val = {}
class2images_test = {}

for label, images in class2images.items():
    class2images_train[label] = [image for image in images if image in train_images]
    class2images_val[label] = [image for image in images if image in val_images]
    class2images_test[label] = [image for image in images if image in test_images]

In [15]:
len(class2images_train['Cardiomegaly'])

1342

In [19]:
class2images_train.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation', 'Pleural Thickening'])

In [35]:
len(class2images_val['Cardiomegaly'])

365

In [16]:
class2images_val.keys()

dict_keys(['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax', 'Pneumonia', 'Fibrosis', 'Edema', 'Consolidation', 'Pleural Thickening'])

In [17]:
len(class2images_test['Cardiomegaly'])

1065

In [18]:
splits_path = "/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/splits"

import os
import pickle

with open(os.path.join(splits_path, "class2images_train.p"), "wb") as f:
    pickle.dump(class2images_train, f)

with open(os.path.join(splits_path, "class2images_val.p"), "wb") as f:
    pickle.dump(class2images_val, f)

with open(os.path.join(splits_path, "class2images_test.p"), "wb") as f:
    pickle.dump(class2images_test, f)

In [1]:
import numpy as np

In [3]:
# read /home/sn666/explainable_ai/LaBo/exp/asso_opt/XRAY_NIH/XRAY_NIH_allshot_fac/concept2cls_selected.npy
concept2cls_selected = np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/XRAY_NIH/XRAY_NIH_allshot_fac/concept2cls_selected.npy", allow_pickle=True)
concept2cls_selected

array([0., 0.])

In [7]:
import torch

select_idx = torch.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/XRAY_NIH/XRAY_NIH_allshot_fac/select_idx.pth")
select_idx

tensor([0, 1])

In [8]:
concept2cls_selected = np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_allshot_fac/concept2cls_selected.npy", allow_pickle=True)
concept2cls_selected

array([3., 3., 5., ..., 4., 0., 0.])

In [5]:
import torch

tens = torch.tensor(
    [
        [
            0.0173,
            0.0176,
            0.0186,
            0.0179,
            0.0187,
            0.0172,
            0.0178,
            0.0174,
            0.0169,
            0.0181,
            0.0181,
            0.0176,
            0.0183,
            0.0174,
        ],
        [
            0.0222,
            0.0221,
            0.0231,
            0.0226,
            0.0229,
            0.0220,
            0.0224,
            0.0222,
            0.0221,
            0.0226,
            0.0222,
            0.0222,
            0.0230,
            0.0223,
        ],
        [
            0.0208,
            0.0207,
            0.0211,
            0.0207,
            0.0222,
            0.0201,
            0.0205,
            0.0204,
            0.0197,
            0.0207,
            0.0205,
            0.0206,
            0.0209,
            0.0206,
        ],
        [
            0.0204,
            0.0203,
            0.0211,
            0.0210,
            0.0212,
            0.0201,
            0.0204,
            0.0203,
            0.0202,
            0.0205,
            0.0202,
            0.0204,
            0.0211,
            0.0204,
        ],
        [
            0.0326,
            0.0329,
            0.0328,
            0.0327,
            0.0327,
            0.0328,
            0.0327,
            0.0329,
            0.0330,
            0.0327,
            0.0326,
            0.0327,
            0.0327,
            0.0328,
        ],
    ]
)

tens

tensor([[0.0173, 0.0176, 0.0186, 0.0179, 0.0187, 0.0172, 0.0178, 0.0174, 0.0169,
         0.0181, 0.0181, 0.0176, 0.0183, 0.0174],
        [0.0222, 0.0221, 0.0231, 0.0226, 0.0229, 0.0220, 0.0224, 0.0222, 0.0221,
         0.0226, 0.0222, 0.0222, 0.0230, 0.0223],
        [0.0208, 0.0207, 0.0211, 0.0207, 0.0222, 0.0201, 0.0205, 0.0204, 0.0197,
         0.0207, 0.0205, 0.0206, 0.0209, 0.0206],
        [0.0204, 0.0203, 0.0211, 0.0210, 0.0212, 0.0201, 0.0204, 0.0203, 0.0202,
         0.0205, 0.0202, 0.0204, 0.0211, 0.0204],
        [0.0326, 0.0329, 0.0328, 0.0327, 0.0327, 0.0328, 0.0327, 0.0329, 0.0330,
         0.0327, 0.0326, 0.0327, 0.0327, 0.0328]])

In [6]:
torch.sigmoid(tens) > 0.5

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True]])

In [10]:
import numpy as np
t1 = np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_allshot_fac_biomedclip/concepts_raw_selected.npy")

In [11]:
t2 = np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_allshot_fac/concepts_raw_selected.npy")

In [13]:
(t1 == t2).all()

True

In [3]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/datasets/XRAY_NIH/concepts/concepts_raw.npy")

array(['Elevation of the hemidiaphragm on the affected side',
       'Displacement of Hilar Structures: The hilar structures might be displaced towards the affected side.',
       'Decreased rib expansion on affected side', ...,
       'Aberrant gastric air pattern',
       'Presence of soft tissue mass in unusual locations',
       'Unilateral Diaphragmatic Elevation'], dtype='<U186')

In [1]:
import json

In [5]:
import os

with open("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/concepts/class2concepts.json", 'r') as f:
    ham10000_concept = json.load(f)

ham10000_concept.keys()

dict_keys(['actinic keratoses', 'basal cell carcinoma', 'benign keratosis-like lesions', 'dermatofibroma', 'melanocytic nevi', 'melanoma', 'vascular lesions'])

In [2]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_specialist_allshot_fac/concepts_raw_selected.npy")

array(['Absence of atypical network',
       'Absence of atypical vascular patterns',
       'Absence of atypical vasculature: Nevi usually do not show unusual blood vessels or structures like those seen in more concerning lesions.',
       ..., 'Yellowish undertones', 'Yellowish-tan color',
       'Yellowness or yellow hue'], dtype='<U231')

In [4]:
np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_specialist_allshot_fac/concepts_raw_selected.npy")

array(['Absence of atypical network',
       'Absence of atypical vascular patterns',
       'Absence of atypical vasculature: Nevi usually do not show unusual blood vessels or structures like those seen in more concerning lesions.',
       ..., 'Yellowish undertones', 'Yellowish-tan color',
       'Yellowness or yellow hue'], dtype='<U231')

In [6]:
import os
import platform

def creation_date(path_to_file):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        return os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            return stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            mtime = stat.st_mtime
            # convert mtime to datetime
            import datetime
            return datetime.datetime.fromtimestamp(mtime)

creation_date("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_specialist_allshot_fac/select_idx.pth")

datetime.datetime(2025, 3, 23, 14, 43, 30, 952196)

In [8]:
import torch
torch.load("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/splits/label_train_all.pth")

tensor([0, 0, 0,  ..., 6, 6, 6])

In [3]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_MOE_allshot_fac/concepts_raw_selected.npy")

array(['"button-like" growth', '1-5 cm in diameter',
       'a cancerous growth that appears as a dark, raised area on the skin',
       ..., 'yellow in color', 'yellow, white, or light brown in color',
       'yellow, white, or tan in color'], dtype='<U109')

In [4]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_MOE_allshot_fac/concepts_raw_selected_specialist.npy")

array(['Absence of atypical network',
       'Absence of atypical vascular patterns',
       'Absence of atypical vasculature: Nevi usually do not show unusual blood vessels or structures like those seen in more concerning lesions.',
       ..., 'Yellowish undertones', 'Yellowish-tan color',
       'Yellowness or yellow hue'], dtype='<U231')

In [1]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000/HAM10000_MOE_allshot_fac/concept2cls_selected_specialist.npy")

array([2., 3., 4., ..., 1., 2., 2.])

In [1]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/concepts_specialist/concepts_raw.npy")

array(['Scaly surface texture', 'Presence of pigmentation variations',
       'Red', ..., 'Hemorrhagic crust', 'Dark red color',
       'Circular pattern'], dtype='<U231')

In [3]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/concepts_specialist/concept2cls.npy")

array([0., 0., 0., ..., 6., 6., 6.])

In [4]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/concepts_specialist/cls_names.npy")

array(['actinic keratoses', 'basal cell carcinoma',
       'benign keratosis-like lesions', 'dermatofibroma',
       'melanocytic nevi', 'melanoma', 'vascular lesions'], dtype='<U29')

In [5]:
concepts_raw = np.load("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/concepts_specialist/concepts_raw.npy")
concepts2cls = np.load("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/concepts_specialist/concept2cls.npy")
cls_names = np.load("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/concepts_specialist/cls_names.npy")

In [21]:
# reconstruct cls2concepts
cls2concepts = {}
for idx, concept in enumerate(concepts_raw):
    cls = cls_names[int(concepts2cls[idx])]
    cls2concepts.setdefault(cls, []).append(concept)
    

In [9]:
len(concepts_raw)

1473

In [19]:
cls_names

array(['actinic keratoses', 'basal cell carcinoma',
       'benign keratosis-like lesions', 'dermatofibroma',
       'melanocytic nevi', 'melanoma', 'vascular lesions'], dtype='<U29')

In [24]:
import json

with open("/home/sn666/explainable_ai/LaBo/datasets/HAM10000/concepts_specialist/class2concepts.json", 'w') as f:
    json.dump(cls2concepts, f)

In [1]:
# iterate through "/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000_moe" and
# rename "concepts_feat_biomedclip.pth" to "concepts_feat_biomedclip_specialist.pth"
# rename "concepts_raw_selected.npy" to "concepts_raw_selected_specialist.npy"
# rename init_weight.pth to init_weight_specialist.pth
# rename select_idx.pth to select_idx_specialist.pth
# rename concept2cls_selected.npy to concept2cls_selected_specialist.npy

import os

dir_path = "/home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe"

for root, dirs, files in os.walk(dir_path):
    for file in files:
        if file == "concepts_feat_biomedclip.pth":
            os.rename(os.path.join(root, file), os.path.join(root, "concepts_feat_biomedclip_specialist.pth"))
        elif file == "concepts_raw_selected.npy":
            os.rename(os.path.join(root, file), os.path.join(root, "concepts_raw_selected_specialist.npy"))
        elif file == "init_weight.pth":
            os.rename(os.path.join(root, file), os.path.join(root, "init_weight_specialist.pth"))
        elif file == "select_idx.pth":
            os.rename(os.path.join(root, file), os.path.join(root, "select_idx_specialist.pth"))
        elif file == "concept2cls_selected.npy":
            os.rename(os.path.join(root, file), os.path.join(root, "concept2cls_selected_specialist.npy"))
        
        # delete the file if it ends with ckpt
        if file.endswith("ckpt"):
            print(f"Deleting {os.path.join(root, file)}")
            os.remove(os.path.join(root, file))
            
        # delete the file if it ends with .py
        if file.endswith(".py"):
            print(f"Deleting {os.path.join(root, file)}")
            os.remove(os.path.join(root, file))

Deleting /home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_2shot_fac/epoch=179-step=360-val_acc=0.4048.ckpt
Deleting /home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_2shot_fac/COVIDX_specialist_2shot_fac.py
Deleting /home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_allshot_fac/epoch=3739-step=198220-val_acc=0.8993.ckpt
Deleting /home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_allshot_fac/COVIDX_specialist_allshot_fac.py
Deleting /home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_4shot_fac/COVIDX_specialist_4shot_fac.py
Deleting /home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_4shot_fac/epoch=4329-step=8660-val_acc=0.6965.ckpt
Deleting /home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_8shot_fac/COVIDX_specialist_8shot_fac.py
Deleting /home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_8shot_fac/epoch=29-step=120-val_acc=0.6879.ckpt
Deleting /home/sn666/

In [6]:
import torch

t = torch.tensor([[109.3589, 109.8273, 110.3519, 110.6970, 109.8260, 111.1373, 109.7937,
         109.7623, 110.9990, 107.5351, 108.2846, 110.6758, 112.2845, 110.9098],
        [113.1080, 115.8277, 113.8450, 114.3301, 114.0114, 114.6241, 113.0033,
         116.5330, 114.6781, 112.1692, 111.6824, 114.1536, 115.3977, 115.2845],
        [114.3033, 114.1362, 114.9378, 114.7914, 116.1840, 115.9934, 114.6936,
         115.2184, 115.2824, 111.6027, 111.7114, 116.3848, 116.6951, 116.5176],
        [112.3054, 113.7064, 114.9036, 114.0560, 114.6381, 113.8668, 113.0195,
         114.0021, 114.3893, 110.9910, 110.8461, 113.8629, 115.9116, 114.5467],
        [ 94.7431,  96.1051,  96.3196,  96.6817,  95.8061,  96.6041,  95.2176,
          97.4514,  96.9019,  93.7244,  93.1725,  95.8856,  98.4823,  97.2439],
        [105.5771, 106.4950, 106.7849, 106.9523, 106.0461, 107.4537, 105.4768,
         106.2668, 106.9937, 103.2721, 103.4381, 106.5455, 107.9018, 107.9317],
        [106.1932, 108.3027, 107.3143, 108.3443, 106.0257, 107.8821, 107.0190,
         106.1485, 107.9012, 103.8438, 103.6752, 106.9307, 109.0232, 107.3046],
        [109.7731, 109.9276, 110.8650, 110.9072, 110.9902, 111.9373, 109.9726,
         110.7781, 111.2193, 107.4310, 107.5453, 110.7651, 112.6154, 112.5757],
        [125.7349, 127.5082, 126.4089, 126.7967, 126.7051, 127.1060, 125.8259,
         127.4090, 127.3131, 124.1353, 124.1697, 127.4946, 128.3987, 127.6741],
        [121.5660, 121.9391, 122.9451, 122.3944, 123.3899, 122.4707, 121.6740,
         122.9352, 123.2288, 120.2047, 120.3970, 123.8938, 124.5418, 124.1515],
        [115.9994, 115.2889, 117.0612, 117.3039, 116.0397, 117.5092, 116.5685,
         115.4288, 117.4246, 113.2959, 113.7217, 116.9447, 118.5482, 117.4871],
        [120.8892, 120.2464, 121.6061, 121.4152, 120.9440, 122.2549, 121.1322,
         120.4210, 122.1735, 118.7919, 119.3483, 122.1947, 123.0234, 122.2943],
        [102.5294, 102.4541, 104.3471, 104.1513, 103.4806, 103.5387, 103.1885,
         103.0852, 104.0967, 100.6881, 200, 103.9974, 106.1850, 104.5798]])

In [23]:
import torch as th
import numpy as np

torch.sigmoid(t * (th.ones([]) * np.log(1 / 550)))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [32]:
th.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000_specialist/HAM10000_specialist_1shot_fac/concepts_feat_biomedclip.pth")

tensor([[ 1.2798e-01,  9.7162e-02, -2.8822e-01,  ...,  5.1127e-02,
          4.5608e-02,  6.0804e-02],
        [ 8.8931e-02,  6.6376e-02, -4.5966e-01,  ..., -1.2169e-01,
         -9.3826e-02,  3.5995e-01],
        [ 6.5682e-02,  1.9440e-01, -1.7688e-01,  ..., -2.2725e-04,
          9.4088e-02,  3.3082e-01],
        ...,
        [ 3.9239e-02, -1.4285e-01, -3.0175e-01,  ..., -7.1901e-02,
          1.0430e-02, -1.1887e-03],
        [ 2.3707e-01, -5.1015e-02, -2.5693e-01,  ...,  9.1313e-02,
          1.9125e-02,  1.4667e-01],
        [ 8.1634e-02, -1.2211e-02, -3.7128e-01,  ...,  9.7777e-02,
          5.3847e-02,  1.4238e-02]])

In [33]:
th.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000_specialist/HAM10000_specialist_4shot_fac/concepts_feat_biomedclip.pth")

tensor([[ 0.0742,  0.0839, -0.2283,  ...,  0.0755,  0.0149,  0.0201],
        [ 0.1251,  0.0341, -0.5358,  ..., -0.0186, -0.0487,  0.3248],
        [ 0.1046,  0.1537, -0.2288,  ...,  0.0065,  0.1354,  0.4110],
        ...,
        [-0.0684,  0.0676, -0.3292,  ...,  0.0093,  0.0017, -0.0189],
        [ 0.2247, -0.0228, -0.2609,  ...,  0.0478, -0.0142,  0.0856],
        [ 0.0531,  0.0160, -0.3553,  ...,  0.0606, -0.0312,  0.0571]])

In [29]:
th.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000_moe/HAM10000_moe_1shot_fac/concepts_feat_biomedclip_specialist.pth")

tensor([[ 1.2798e-01,  9.7162e-02, -2.8822e-01,  ...,  5.1127e-02,
          4.5608e-02,  6.0804e-02],
        [ 8.8931e-02,  6.6376e-02, -4.5966e-01,  ..., -1.2169e-01,
         -9.3826e-02,  3.5995e-01],
        [ 6.5682e-02,  1.9440e-01, -1.7688e-01,  ..., -2.2725e-04,
          9.4088e-02,  3.3082e-01],
        ...,
        [ 3.9239e-02, -1.4285e-01, -3.0175e-01,  ..., -7.1901e-02,
          1.0430e-02, -1.1887e-03],
        [ 2.3707e-01, -5.1015e-02, -2.5693e-01,  ...,  9.1313e-02,
          1.9125e-02,  1.4667e-01],
        [ 8.1634e-02, -1.2211e-02, -3.7128e-01,  ...,  9.7777e-02,
          5.3847e-02,  1.4238e-02]])

In [35]:
t = th.tensor([[2.3621, 2.3406, 2.3575, 2.3725, 2.3156, 2.3558, 2.3271, 2.3084, 2.3553,
         2.2662, 2.2598, 2.3289, 2.3674, 2.3583],
        [2.4127, 2.4257, 2.4339, 2.4470, 2.4186, 2.4295, 2.4045, 2.4221, 2.4285,
         2.3519, 2.3363, 2.4094, 2.4509, 2.4452],
        [2.5030, 2.4677, 2.5301, 2.5077, 2.4978, 2.5062, 2.4829, 2.4743, 2.5187,
         2.4769, 2.4875, 2.5212, 2.5332, 2.5274],
        [2.6965, 2.7166, 2.6898, 2.6808, 2.7578, 2.7118, 2.6627, 2.7354, 2.6748,
         2.6296, 2.6073, 2.7296, 2.6954, 2.7409],
        [2.5072, 2.4659, 2.5151, 2.4929, 2.5398, 2.4946, 2.4896, 2.5019, 2.4928,
         2.4610, 2.4555, 2.5336, 2.5323, 2.5201],
        [2.4562, 2.4358, 2.4580, 2.4616, 2.4389, 2.4647, 2.4265, 2.4356, 2.4570,
         2.3711, 2.3732, 2.4444, 2.4662, 2.4844],
        [2.6118, 2.6428, 2.6021, 2.6119, 2.6306, 2.6083, 2.5665, 2.6230, 2.5973,
         2.5391, 2.5200, 2.6208, 2.6206, 2.6346],
        [2.9508, 2.9103, 2.9427, 2.9366, 2.9428, 2.9455, 2.9305, 2.9200, 2.9375,
         2.8713, 2.8808, 2.9648, 2.9560, 2.9572],
        [2.5935, 2.5496, 2.5785, 2.5866, 2.5531, 2.6019, 2.5640, 2.5531, 2.5823,
         2.4961, 2.4933, 2.5721, 2.5886, 2.6083],
        [2.6540, 2.6451, 2.6527, 2.6655, 2.6403, 2.6775, 2.6355, 2.6358, 2.6564,
         2.5687, 2.5708, 2.6471, 2.6646, 2.6827],
        [2.8636, 2.8683, 2.8479, 2.8570, 2.8560, 2.8686, 2.8281, 2.8517, 2.8523,
         2.7865, 2.7832, 2.8650, 2.8590, 2.8816],
        [2.8310, 2.8456, 2.8254, 2.8235, 2.8339, 2.8308, 2.7994, 2.8474, 2.8268,
         2.7845, 2.7803, 2.8474, 2.8301, 2.8509],
        [2.9812, 2.9862, 2.9656, 2.9827, 2.9581, 2.9855, 2.9509, 2.9626, 2.9778,
         2.9217, 2.9185, 2.9818, 2.9796, 2.9928],
        [2.7086, 2.6652, 2.7088, 2.7231, 2.6806, 2.7274, 2.7137, 2.6810, 2.7203,
         2.6299, 2.6305, 2.7028, 2.7361, 2.7166]])

In [36]:
torch.sigmoid(t)

tensor([[0.9139, 0.9122, 0.9135, 0.9147, 0.9102, 0.9134, 0.9111, 0.9096, 0.9134,
         0.9060, 0.9055, 0.9112, 0.9143, 0.9136],
        [0.9178, 0.9188, 0.9194, 0.9203, 0.9182, 0.9190, 0.9172, 0.9185, 0.9190,
         0.9131, 0.9118, 0.9175, 0.9206, 0.9202],
        [0.9244, 0.9218, 0.9262, 0.9247, 0.9240, 0.9246, 0.9229, 0.9223, 0.9254,
         0.9225, 0.9233, 0.9256, 0.9264, 0.9260],
        [0.9368, 0.9380, 0.9364, 0.9359, 0.9404, 0.9377, 0.9348, 0.9391, 0.9355,
         0.9327, 0.9313, 0.9388, 0.9368, 0.9394],
        [0.9246, 0.9217, 0.9252, 0.9236, 0.9269, 0.9238, 0.9234, 0.9243, 0.9236,
         0.9214, 0.9210, 0.9265, 0.9264, 0.9255],
        [0.9210, 0.9195, 0.9211, 0.9214, 0.9197, 0.9216, 0.9188, 0.9195, 0.9211,
         0.9146, 0.9148, 0.9202, 0.9217, 0.9230],
        [0.9316, 0.9336, 0.9310, 0.9316, 0.9328, 0.9314, 0.9287, 0.9323, 0.9307,
         0.9268, 0.9255, 0.9322, 0.9322, 0.9331],
        [0.9503, 0.9484, 0.9499, 0.9496, 0.9499, 0.9501, 0.9493, 0.9488, 0.9497,
  

In [4]:
import numpy as np

concepts = np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000_moe/HAM10000_moe_allshot_fac/concepts_raw_selected_specialist.npy")

In [13]:
import torch as th

ids_selected_spec = th.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000_moe/HAM10000_moe_allshot_fac/select_idx_specialist.pth")
ids_selected_gen = th.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/HAM10000_moe/HAM10000_moe_allshot_fac/select_idx.pth")

In [6]:
concepts

array(['Absence of atypical network',
       'Absence of atypical vascular patterns',
       'Absence of atypical vasculature: Nevi usually do not show unusual blood vessels or structures like those seen in more concerning lesions.',
       ..., 'Yellowish undertones', 'Yellowish-tan color',
       'Yellowness or yellow hue'], dtype='<U231')

In [17]:
selected_spec = concepts[ids_selected_spec]

In [18]:
selected_gen = concepts[ids_selected_gen]

In [14]:
ids_selected_gen

tensor([ 234, 1002, 1043, 1292,  962, 1004,  568, 1003,  763,  973,  535, 1290,
        1005, 1006, 1044,  231,   53,  885,  233,  972,   29,  767,  487,  146,
         955,  396, 1001,  961,  229,  232,   26, 1098,  538,  765,  958,  971,
         764,  130,  912,  883,  390,  140,  256,  987, 1262,  628, 1084, 1264,
        1034, 1000,  127, 1029,   56,  905, 1141,  624, 1271,  580,  660,  606,
          41,  630, 1294, 1131,  223, 1178,  878, 1033, 1295, 1020,  267,  582,
         603,  249,  159,  204,  755,   74,  829,  804,   28, 1184,  772,  134,
         880,  684,   79,  690,  985,  455, 1307,  819,  327,   78,  877,  738,
        1097,  651,  692,  748, 1099,  604, 1196,  532, 1093,  279, 1270,  570,
        1100,  422,  107,  208,  243,  282,  575,  241,  408, 1308,  557,  616,
         573,  548, 1026,  799,   36,  428,  418,  558,   39,  177,  789, 1267,
        1132,  366,  830,  950,  369,  184,  969,  374,  670,  864,  381,  559,
         640,  990,  442,  964,  645, 11

In [23]:
# get the most common words in selected_spec
from collections import Counter

words = []
for concept in selected_spec:
    words.extend(concept.split())

# remove stopwords
from nltk.corpus import stopwords
words = [word for word in words if word not in stopwords.words("english")]

Counter(words).most_common()

[('Presence', 24),
 ('brown', 22),
 ('areas', 19),
 ('color', 19),
 ('distribution', 16),
 ('pigmentation', 16),
 ('lesion', 16),
 ('Color', 14),
 ('pattern', 12),
 ('within', 11),
 ('Pigment', 11),
 ('Dark', 11),
 ('network', 11),
 ('Erythematous', 10),
 ('often', 10),
 ('texture', 10),
 ('vessels', 10),
 ('pigment', 10),
 ('structures', 10),
 ('skin', 9),
 ('variation', 9),
 ('patches', 9),
 ('blood', 9),
 ('lines', 9),
 ('Light', 9),
 ('lesion.', 9),
 ('may', 8),
 ('Brown', 8),
 ('shape', 8),
 ('white', 8),
 ('Focal', 7),
 ('red', 7),
 ('appearance', 7),
 ('A', 7),
 ('Central', 7),
 ('ulceration', 7),
 ('hue', 7),
 ('indicating', 7),
 ('surface', 7),
 ('Lesion', 7),
 ('dots', 7),
 ('due', 6),
 ('Areas', 6),
 ('pink', 6),
 ('Pigmented', 6),
 ('surrounding', 6),
 ('spots', 6),
 ('globules', 6),
 ('structure', 6),
 ('depigmentation', 6),
 ('Network:', 6),
 ('Uniform', 6),
 ('Vascular', 6),
 ('typically', 5),
 ('pigmented', 5),
 ('Ulceration', 5),
 ('lesion,', 5),
 ('presence', 5),
 ('S

In [24]:
# get the most common words in selected_spec
from collections import Counter

words = []
for concept in selected_gen:
    words.extend(concept.split())

# remove stopwords
from nltk.corpus import stopwords
words = [word for word in words if word not in stopwords.words("english")]

Counter(words).most_common()

[('lesion', 38),
 ('brown', 33),
 ('texture', 23),
 ('Presence', 22),
 ('shape', 19),
 ('color', 19),
 ('surface', 15),
 ('areas', 15),
 ('Light', 14),
 ('within', 14),
 ('Brown', 14),
 ('Surface', 13),
 ('Dark', 12),
 ('red', 11),
 ('structures', 10),
 ('Diameter', 10),
 ('Lesion', 10),
 ('Red', 9),
 ('Diameter:', 9),
 ('Scaly', 8),
 ('often', 8),
 ('presence', 8),
 ('pattern', 8),
 ('lesion,', 8),
 ('size', 8),
 ('lesion.', 8),
 ('dots', 8),
 ('patches', 7),
 ('due', 7),
 ('colors', 7),
 ('pigmentation', 7),
 ('larger', 7),
 ('6mm', 7),
 ('globules', 7),
 ('diameter', 6),
 ('scaly', 6),
 ('white', 6),
 ('skin', 6),
 ('keratin', 6),
 ('Size', 6),
 ('lines', 6),
 ('Asymmetry:', 6),
 ('may', 6),
 ('multiple', 6),
 ('black,', 6),
 ('Uniform', 6),
 ('White', 5),
 ('crusty', 5),
 ('A', 5),
 ('Pink', 5),
 ('bleeding', 5),
 ('less', 5),
 ('half', 5),
 ('tan', 5),
 ('Color', 5),
 ('vessels', 5),
 ('The', 5),
 ('oval', 5),
 ('hue', 5),
 ('black', 5),
 ('Crusty', 4),
 ('scales', 4),
 ('scaling'

In [1]:
# look through /home/sn666/explainable_ai/LaBo/cfg/asso_opt/COVIDX_moe and if the file name contains "HAM10000", replace it with COVIDX
import os

dir_path = "/home/sn666/explainable_ai/LaBo/cfg/asso_opt/COVIDX_moe"
for f in os.listdir(dir_path):
    if "HAM10000" in f:
        new_f = f.replace("HAM10000", "COVIDX")
        print(f"Renaming {f} to {new_f}")
        os.rename(os.path.join(dir_path, f), os.path.join(dir_path, new_f))

Renaming HAM10000_moe_16shot_fac.py to COVIDX_moe_16shot_fac.py
Renaming HAM10000_moe_allshot_fac.py to COVIDX_moe_allshot_fac.py
Renaming HAM10000_base.py to COVIDX_base.py
Renaming HAM10000_moe_2shot_fac.py to COVIDX_moe_2shot_fac.py
Renaming HAM10000_moe_8shot_fac.py to COVIDX_moe_8shot_fac.py
Renaming HAM10000_moe_4shot_fac.py to COVIDX_moe_4shot_fac.py
Renaming HAM10000_moe_1shot_fac.py to COVIDX_moe_1shot_fac.py


In [2]:
import numpy as np

np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_1shot_fac/concepts_raw_selected.npy")

array(['Abnormal white patches', 'Absence of Cavitation',
       'Absence of Consolidation', 'Absence of Effusion',
       'Absence of Infiltrates', 'Absence of Masses',
       'Absence of Masses or Nodules', 'Absence of Nodules',
       'Absence of Pleural Effusion', 'Absence of air bronchograms',
       'Absence of atelectasis',
       'Absence of calcifications in lung tissue',
       'Absence of cavitation', 'Absence of consolidation',
       'Absence of fluid or pleural effusion',
       'Absence of foreign bodies', 'Absence of lesions',
       'Absence of lobar consolidation', 'Absence of nodules',
       'Absence of pleural effusion',
       'Absence of significant lymphadenopathy',
       'Accompanying pleural effusion', 'Air Bronchogram',
       'Air Bronchograms', 'Air bronchogram', 'Air bronchogram presence',
       'Air bronchogram sign', 'Air bronchograms',
       'Air bronchograms visibility', 'Air space consolidation',
       'Air-space consolidation', 'Airspace consolid

In [3]:
np.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_1shot_fac/concepts_raw_selected_specialist.npy")

array(['Abnormal white patches', 'Absence of Cavitation',
       'Absence of Consolidation', 'Absence of Effusion',
       'Absence of Infiltrates', 'Absence of Masses',
       'Absence of Masses or Nodules', 'Absence of Nodules',
       'Absence of Pleural Effusion', 'Absence of air bronchograms',
       'Absence of atelectasis',
       'Absence of calcifications in lung tissue',
       'Absence of cavitation', 'Absence of consolidation',
       'Absence of fluid or pleural effusion',
       'Absence of foreign bodies', 'Absence of lesions',
       'Absence of lobar consolidation', 'Absence of nodules',
       'Absence of pleural effusion',
       'Absence of significant lymphadenopathy',
       'Accompanying pleural effusion', 'Air Bronchogram',
       'Air Bronchograms', 'Air bronchogram', 'Air bronchogram presence',
       'Air bronchogram sign', 'Air bronchograms',
       'Air bronchograms visibility', 'Air space consolidation',
       'Air-space consolidation', 'Airspace consolid

In [7]:
import torch

s_gen = torch.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_1shot_fac/select_idx.pth")

In [8]:
s_spec = torch.load("/home/sn666/explainable_ai/LaBo/exp/asso_opt/COVIDX_moe/COVIDX_moe_1shot_fac/select_idx_specialist.pth")

In [10]:
s_gen.unique()

tensor([  1,  11,  12,  22,  24,  26,  47,  49,  50,  54,  56,  57,  58,  59,
         61,  62,  64,  67,  70,  71,  75,  76,  78,  81,  82,  87,  94,  98,
         99, 102, 103, 107, 108, 110, 111, 117, 119, 127, 135, 137, 138, 139,
        143, 146, 147, 149, 150, 155, 156, 157, 158, 160, 161, 164, 167, 168,
        169, 170, 176, 177, 180, 187, 197, 201, 202, 203, 205, 212, 213, 218,
        222, 226, 227, 229, 233, 236, 239, 244, 248, 253, 254, 257, 258, 260,
        265, 267, 268, 281, 282, 285, 286, 287, 291, 293, 300, 304, 305, 306,
        307, 308, 310, 311, 312, 314, 319, 320, 321, 322, 323, 324, 325, 327,
        328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 347, 349, 357, 358,
        364, 368, 369, 392, 394, 395, 397, 400, 401, 405, 406, 407, 408, 409,
        421, 425, 434, 436, 438, 442, 443, 444, 445, 462, 464, 469, 472, 475,
        476, 477, 482, 485, 487, 488, 489, 496, 497, 498, 509, 514, 515, 520,
        536, 539, 540, 541, 555, 556, 557, 558, 559, 563, 566, 5

In [11]:
s_spec.unique()

tensor([  3,   6,   7,   8,  16,  18,  19,  20,  21,  22,  23,  27,  29,  30,
         31,  38,  45,  50,  51,  58,  61,  62,  63,  68,  72,  74,  81,  85,
         91,  93,  94,  96,  97,  98,  99, 104, 107, 111, 116, 117, 118, 119,
        120, 121, 122, 123, 124, 125, 127, 135, 136, 137, 138, 139, 140, 142,
        143, 144, 145, 147, 148, 149, 156, 157, 159, 160, 161, 167, 168, 171,
        172, 175, 176, 180, 181, 192, 195, 196, 199, 204, 205, 206, 217, 223,
        226, 228, 229, 240, 250, 251, 252, 253, 260, 276, 284, 287, 290, 292,
        294, 295, 298, 299, 304, 305, 309, 314, 320, 327, 331, 337, 338, 341,
        342, 343, 345, 346, 356, 357, 368, 372, 374, 381, 382, 384, 385, 386,
        387, 388, 389, 390, 391, 396, 397, 398, 400, 403, 404, 409, 410, 414,
        415, 416, 418, 419, 429, 436, 439, 440, 444, 446, 447, 451, 453, 454,
        455, 456, 457, 461, 464, 465, 466, 469, 471, 475, 476, 478, 483, 487,
        488, 495, 505, 506, 511, 514, 515, 517, 519, 521, 524, 5